## XGBOOSTİNG regressor : 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams["figure.figsize"] = (9,5)
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)

In [2]:
df=pd.read_csv("car_dataset.csv")
df.head()

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,ritz,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,swift,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0


In [3]:
df["vehicle_age"]=2021-df.Year

In [4]:
df.drop(columns=["Year"], inplace=True)

### Train test split and Ordinalencoder : 

In [5]:
X=df.drop("Selling_Price", axis=1)
y=df.Selling_Price

from sklearn.preprocessing import OrdinalEncoder

cat = X.select_dtypes("object").columns
enc = OrdinalEncoder()
X[cat] = enc.fit_transform(X[cat])

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2, random_state=5)

In [7]:
!pip install xgboost

### Default Modell : 

In [8]:
from xgboost import XGBRegressor

xgb_model = XGBRegressor(random_state = 101)
xgb_model.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=4,
             num_parallel_tree=1, predictor='auto', random_state=101,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

### Metrics : 

In [9]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
def train_val(model, X_train, y_train, X_test, y_test):
    
    y_pred = model.predict(X_test)
    y_train_pred = model.predict(X_train)
    
    scores = {"train": {"R2" : r2_score(y_train, y_train_pred),
    "mae" : mean_absolute_error(y_train, y_train_pred),
    "mse" : mean_squared_error(y_train, y_train_pred),                          
    "rmse" : np.sqrt(mean_squared_error(y_train, y_train_pred))},
    
    "test": {"R2" : r2_score(y_test, y_pred),
    "mae" : mean_absolute_error(y_test, y_pred),
    "mse" : mean_squared_error(y_test, y_pred),
    "rmse" : np.sqrt(mean_squared_error(y_test, y_pred))}}
    
    return pd.DataFrame(scores)

In [10]:
train_val(xgb_model, X_train, y_train, X_test, y_test)

,train,test
R2,0.999999,0.971793
mae,0.004302,0.503537
mse,0.000035,0.757107
rmse,0.005933,0.870119


In [24]:
# min_child_weight and gamma değerini arttırarak skorlara bakalım overfitnge ciddi etkisi var : 
deneme = XGBRegressor(random_state = 101, min_child_weight = 4, gamma = 1.5)
deneme.fit(X_train, y_train)
train_val(deneme, X_train, y_train, X_test, y_test)

,train,test
R2,0.983334,0.951796
mae,0.329306,0.580420
mse,0.423878,1.293838
rmse,0.651060,1.137470


### Önemli parametreler : 

gamma, min_child_weight, reg_alpha, reg_lambda, max_depth, n_estimators, learning_rate

In [25]:
xgb_model.get_params()

{'objective': 'reg:squarederror',
 'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'enable_categorical': False,
 'gamma': 0,
 'gpu_id': -1,
 'importance_type': None,
 'interaction_constraints': '',
 'learning_rate': 0.300000012,
 'max_delta_step': 0,
 'max_depth': 6,
 'min_child_weight': 1,
 'missing': nan,
 'monotone_constraints': '()',
 'n_estimators': 100,
 'n_jobs': 4,
 'num_parallel_tree': 1,
 'predictor': 'auto',
 'random_state': 101,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'subsample': 1,
 'tree_method': 'exact',
 'validate_parameters': 1,
 'verbosity': None}

In [26]:
from sklearn.model_selection import cross_validate

model = XGBRegressor(random_state=101, objective="reg:squarederror")
scores = cross_validate(model, X_train, y_train, scoring=['r2', 
            'neg_mean_absolute_error','neg_mean_squared_error','neg_root_mean_squared_error'], cv =10)
pd.DataFrame(scores).iloc[:, 2:].mean()

test_r2                             0.900763
test_neg_mean_absolute_error       -0.732499
test_neg_mean_squared_error        -2.789084
test_neg_root_mean_squared_error   -1.435410
dtype: float64

### GridSearchCV : 

In [28]:
n_estimators = [100, 200, 300, 350, 400, 450, 500]
learning_rate = [0.01, 0.05, 0.1, 0.2]
gamma = [0, 1, 2, 3, 3.5, 4, 4.5]
min_child_weight = [1, 2, 3, 4, 5, 7]

param_grid = {"n_estimators":n_estimators, 'max_depth':[3,5,7], "learning_rate": learning_rate,
             "gamma" : gamma, "min_child_weight" : min_child_weight}


model = XGBRegressor(random_state=101, objective="reg:squarederror")

from sklearn.model_selection import GridSearchCV

grid_model = GridSearchCV(estimator=model,
                          param_grid=param_grid,
                          scoring='neg_root_mean_squared_error',
                          cv=10,
                          n_jobs = -1)

grid_model.fit(X_train, y_train)

GridSearchCV(cv=10,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    enable_categorical=False, gamma=None,
                                    gpu_id=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,...
                                    random_state=101, reg_alpha=None,
                                    reg_lambda=None, scale_pos_weight=None,
                                    subsample=None, tree_method=None,
                                    validate_parameters=None, verbosity=None)

In [29]:
grid_model.best_params_

{'gamma': 2,
 'learning_rate': 0.2,
 'max_depth': 7,
 'min_child_weight': 3,
 'n_estimators': 100}

In [30]:
train_val(grid_model, X_train, y_train, X_test, y_test)

,train,test
R2,0.987958,0.940846
mae,0.324241,0.616782
mse,0.306274,1.587736
rmse,0.553421,1.260054


In [31]:
train_val(xgb_model, X_train, y_train, X_test, y_test)

,train,test
R2,0.999999,0.971793
mae,0.004302,0.503537
mse,0.000035,0.757107
rmse,0.005933,0.870119
